# Shapley's Additive Explanations

This Jupyter notebook explains how SHAP library can be used to provide explanations for a black box model. We will use the TreeSHAP and KernelSHAP modules to provide explanations for a Gradient Boosted Decision Tree  (GBDT) for tabular data and neural network. We will compare explanations provided by LIME and SHAP. 


## **1. Import required libraries**

In [ ]:

import pandas as pd
import numpy as numpy
import shap
shap.initjs()
import matplotlib.pyplot as plt
plt.style.use('default') 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, f1_score
import warnings
warnings.simplefilter("ignore", category=UserWarning)




##  **2. Load Dataset**

We will use the Heart Failure Prediction Dataset from Kaggle available from [this download link](https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset). The Behavioral Risk Factor Surveillance System (BRFSS) is a health-related telephone survey that is collected annually by the CDC. Each year, the survey collects responses from over 400,000 Americans on health-related risk behaviors, chronic health conditions, and the use of preventative services. It has been conducted every year since 1984. 

This dataset contains 3 files:

1. **diabetes _ 012 _ health _ indicators _ BRFSS2015.csv** is a clean dataset of 253,680 survey responses to the CDC's BRFSS2015. The target variable Diabetes_012 has 3 classes. 0 is for no diabetes or only during pregnancy, 1 is for prediabetes, and 2 is for diabetes. There is class imbalance in this dataset. This dataset has 21 feature variables

2. **diabetes _ binary _ 5050split _ health _ indicators _ BRFSS2015.csv** is a clean dataset of 70,692 survey responses to the CDC's BRFSS2015. It has an equal 50-50 split of respondents with no diabetes and with either prediabetes or diabetes. The target variable Diabetes_binary has 2 classes. 0 is for no diabetes, and 1 is for prediabetes or diabetes. This dataset has 21 feature variables and is balanced.

3. **diabetes _ binary _ health _ indicators _ BRFSS2015.csv** is a clean dataset of 253,680 survey responses to the CDC's BRFSS2015. The target variable Diabetes_binary has 2 classes. 0 is for no diabetes, and 1 is for prediabetes or diabetes. This dataset has 21 feature variables and is not balanced.


For a start, let's use subset 2 which is the balanced-binary dataset. 

In [ ]:
df = pd.read_csv('../datasets/diabetes-health-indicators/diabetes_binary_5050split_health_indicators_BRFSS2015.csv')
df.head()

X = df.drop(columns=['Diabetes_binary'])
y = df['Diabetes_binary']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gbdt_model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)

gbdt_model.fit(X_train, y_train)



# Predict probabilities
y_pred_proba = gbdt_model.predict_proba(X_test)[:, 1]
X_test = pd.DataFrame(X_test, columns=X_train.columns)
y_pred = gbdt_model.predict(X_test)

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_proba)
f1 = f1_score(y_test, y_pred)
print(f"AUC: {auc:.4f}, F1: {f1:.4f}")



## **4. Global Model explanations predictions using TreeSHAP**

In [ ]:
shap_values = shap.TreeExplainer(gbdt_model).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type='bar')

1.	Top Features Driving Predictions
	-	GenHlth (General Health) is the most influential predictor of diabetes in your model. This suggests that a person’s self-reported general health has a strong correlation with diabetes risk.
	-	BMI (Body Mass Index) and Age - this  aligns with medical knowledge that higher BMI and older age increase diabetes risk.
	-	HighBP (High Blood Pressure) and HighChol (High Cholesterol)- this  reinforces known comorbidities of diabetes.
2.	Moderately Important Features
	-	Income and Sex p
	-	HeartDiseaseorAttack - makes sense since cardiovascular conditions often co-occur with diabetes.
	-	CholCheck (Cholesterol Check Frequency) likely indicates whether people who get regular checkups are at a higher or lower risk.
3.	Less Influential Features
	-	Features like DiffWalk (Difficulty Walking), MentHlth (Mental Health), Veggies (Vegetable Consumption), and Stroke have smaller SHAP values, indicating they have a weaker effect on diabetes prediction.
	-	PhysActivity (Physical Activity) has a surprisingly low impact, which could mean that in this dataset, physical activity is not as strong a differentiator for diabetes risk as expected.
	-	NoDocbcCost (No Doctor Visit Due to Cost) and AnyHealthcare appear at the bottom, meaning they contribute very little to predictions.

We typycall use mean absolute values of SHAP to illustrate feature importance, regardless of their positive or negative effect.

In [ ]:
# Generate summary plot
shap.summary_plot(shap_values, X_train)

1. **X-Axis (SHAP Values: Impact on Model Output)**
	-	A negative SHAP value (left side) means that the feature lowers the diabetes risk.
	-	A positive SHAP value (right side) means that the feature increases the diabetes risk.
	-	Points close to 0 indicate low impact.
	
2.	**Color Gradient (Feature Value: Blue → Red)**
	-	Blue = Lower feature values
	-	Red = Higher feature values
	-	This helps in understanding whether high or low feature values push the model towards predicting diabetes.


Let's dive deeper:

1. **General Health (GenHlth) is the Most Important Feature**
	-	Higher values of GenHlth (poor general health) increase diabetes risk (red points on the right).
	-	Lower values of GenHlth (good health) decrease diabetes risk (blue points on the left).

2. **BMI and Age are Strong Predictors**
	-	Higher BMI (red points) is associated with a higher risk (positive SHAP values).
	-	Lower BMI (blue points) is linked to lower risk.
	-	Age follows the same pattern, with older individuals having a higher risk.

3. **High Blood Pressure (HighBP) and High Cholesterol (HighChol) Are Significant**
	-	HighBP: High values (red) increase diabetes risk, while low values (blue) lower it.
	-	HighChol: Similarly, people with high cholesterol levels are more likely to have diabetes.

## **5. Local explanations for specific instance using SHAP**

In addition to providing global explanations, SHAP also allows instance-specific explanations. Here we are applying SHAP to explain the 10th patient data from the test set:

In [1]:
# Select the 10th example from the test set
index = 9
example = X_test.iloc[index]
display(example)
print(f"True label: {y_test.iloc[index]}")
print(f"Predicted probability: {gbdt_model.predict_proba([example])[0, 1]}")
print(f"Predicted class: {gbdt_model.predict([example])[0]}")

# Get SHAP values for the 10th example
shap_values_single = shap.TreeExplainer(gbdt_model).shap_values(example)

# Plot the SHAP values for the 10th example
shap.force_plot(shap.TreeExplainer(gbdt_model).expected_value, shap_values_single, example)

NameError: name 'X_test' is not defined

### **5.1 How do we inteprete these results?**

1.	Base Value (Expected Value):
    - 	This is the average model prediction across the dataset.
    -  	In this case, it is approximately -0.08087.
2.	Final Prediction (f(x)):
    -	The final model prediction for this sample is -3.01, which is lower than the base value.
    -	Since SHAP values explain how features push the prediction higher (towards diabetes) or lower (away from diabetes), a negative prediction likely means a low probability of diabetes for this individual.
3.	Features that Decreased the Prediction (Blue Arrows - Left Side)
    -	GenHlth = 1 (Good General Health)
    -	BMI = 18 (Low BMI)
    -	HighBP = 0 (No High Blood Pressure)
    -	HighChol = 0 (No High Cholesterol)
    -	Income = 8 (High Income)

The <span style="color:blue">blue features</span> pushes the predictions lower, while the <span style="color:red">red features</span> push the predictions slightly higher, but was not strong enough to overcome other health-protective features. In other words:

-	This individual is predicted to have a low diabetes risk, as indicated by the negative final prediction (-3.01).
-	The most influential features driving the low prediction are:
-	Good General Health
-	Low BMI
-	No High Blood Pressure
-	No High Cholesterol
-	High Income
-	The only feature slightly increasing the risk is Age = 10, but its effect is weak compared to the protective features.

### **5.2 Compare this with LIME**


In [ ]:
import lime
import lime.lime_tabular
import warnings
warnings.simplefilter("ignore", category=FutureWarning)


explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, mode='regression', feature_names=X_train.columns.tolist(), discretize_continuous=True)

# Pick the 10th instance from the test data
exp = explainer.explain_instance(example, gbdt_model.predict, num_features=21) 
display(example) 
exp.show_in_notebook()


### **5.3 Intepretation of plots**
- Both methods recognize GenHlth (General Health), BMI, HighBP, HighChol, and Age as key predictors for diabetes risk.
    -  Both show that good general health, lower BMI, and absence of high blood pressure push the prediction towards a lower diabetes risk.

- Both differentiate between positive and negative influences
	
- Both highlight the impact of Age and Health Conditions
	-	In both visualizations, Age (10 years old) has a small positive influence on increasing diabetes risk.


Generally speaking,

| Method | Advantages | Disadvantages |
|--------|------------|--------------|
| **LIME** (Local Interpretable Model-agnostic Explanations) | - Model-agnostic and works with any black-box model.  <br> - Provides human-interpretable explanations.  <br> - Faster computation compared to SHAP.  <br> - Can be used with both tabular and non-tabular data (e.g., images, text). | - Relies on surrogate models, which may not always faithfully represent the actual model's decision boundary. <br> - Sensitive to sampling and kernel settings. <br> - Explanations can vary between runs due to random sampling. <br> - May struggle with complex interactions between features. |
| **SHAP** (SHapley Additive exPlanations) | - Based on solid game-theoretic foundations, ensuring consistency and fairness. <br> - Provides global and local explanations. <br> - More stable and reliable explanations than LIME. <br> - Captures feature interactions effectively. | - Computationally expensive, especially for complex models. <br> - Requires a large number of model evaluations. <br> - Kernel SHAP (model-agnostic version) can be slow on large datasets. <br> - Difficult to interpret when many features are involved. |
